In [3]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Carica il modello pre-addestrato GPT-3
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Testo di input (esempio di referto medico)
input_text = "Paziente maschio di 45 anni con anamnesi di ipertensione. Ha presentato sintomi di"

# Tokenizza l'input
input_ids = tokenizer.encode(input_text, return_tensors='tf')

# Genera testo sintetico
output = model.generate(input_ids, max_length=150, num_return_sequences=1)

# Decodifica e stampa il risultato
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Paziente maschio di 45 anni con anamnesi di ipertensione. Ha presentato sintomi di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di sunt


In [5]:
generated_text

'Paziente maschio di 45 anni con anamnesi di ipertensione. Ha presentato sintomi di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di suntur di sunt'